In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('../Style.mplstyle')
import pandas as pd
import os
import sys
import scipy
sys.path.append('..')
from read_data_functions import *
from plot_functions import *
from calculations import *
import warnings
warnings.filterwarnings('ignore')

In [2]:
parent_path = '../../../../'
path = 'L:/PG-Nanoteknologi/PROJEKTER/2024 Laura og Nan/cph central and nørreport measurements/'

NS1 = read_SMPS(f'{path}Station 1/', parent_path, [-1, 0])
OPS1 = read_OPS(f'{path}Station 1/OPS/', parent_path, 0)
MA1 = read_csv_BC(f'{path}Station 1/Ma200/', parent_path, -1)
NS2 = read_SMPS(f'{path}Station 2/', parent_path, [-1, 0])
OPS2 = read_OPS(f'{path}Station 2/OPS/', parent_path, 0)
MA2 = read_csv_BC(f'{path}Station 2/Ma200/', parent_path, -1)
ACSM = read_csv(f'{path}Station 1/Timeseries/', parent_path, 't_base', None, 1)
LCS = read_csv(f'{path}Korrigeret low cost sensor', parent_path, 'Time', 0, 0)

In [3]:
NS_dict_keys = ['2024-12-09_NanoScan', '2024-12-10_NanoScan', '2024-12-11_NanoScan', '2024-12-12_NanoScan', 
                '2024-12-13_NanoScan', '2024-12-16_NanoScan', '2024-12-17_NanoScan']
NS_bins = NS1['2024-12-06_NanoScan'].keys()[3:16].to_list()

OPS1_dict_keys = ['OPS1-CPH-d1', 'OPS1-CPH-d2', 'OPS1-CPH-d3', 'OPS1-CPH-d4', 'OPS1-CPH-d5', 'OPS1-NP-d1', 'OPS1-NP-d2']
OPS2_dict_keys = ['OPS2-CPH-d1', 'OPS2-CPH-d2', 'OPS2-CPH-d3', 'OPS2-CPH-d4', 'OPS2-CPH-d5', 'OPS2-NP-d1', 'OPS2-NP-d2']
OPS_cut_point = [0.300, 0.374, 0.465, 0.579, 0.721, 0.897, 1.117, 1.391, 1.732, 2.156, 2.685, 3.343, 4.162, 5.182, 6.451, 8.031, 10.000]
OPS_keys = OPS1['OPS1-HEPA'].keys()[1:17].to_list()

MA1_dict_keys = ['MA200-0369_000301200101', 'MA200-0369_241210105000', 'MA200-0369_241211095501', 'MA200-0369_241212094705', 
                 'MA200-0369_241213094000', 'MA200-0369_241216103300', 'MA200-0369_241217094901']
MA2_dict_keys = ['MA200-0285_241209120400', 'MA200-0285_241210105200', 'MA200-0285_241211093400', 'MA200-0285_241212093100', 
                 'MA200-0285_241213092900', 'MA200-0285_241216102104', 'MA200-0285_241217101400']
MA1['MA200-0369_241217094901'] = pd.concat([MA1['MA200-0369_241217094901'], MA1['MA200-0369_241217101259']], ignore_index=True)

ACSM_keys = ['10-12-2024', '11-12-2024', '12-12-2024', '13-12-2024']

LCS1_dict_keys = ['2024-12-09 DG-0159', '2024-12-10 DG-0159', '2024-12-11 DG-0159', '2024-12-12 DG-0159',
                  '2024-12-13 DG-0159', '2024-12-16 DG-0159', '2024-12-17 DG-0159']
LCS2_dict_keys = ['2024-12-09 DG-0158', '2024-12-10 DG-0158', '2024-12-11 DG-0158', '2024-12-12 DG-0158',
                  '2024-12-13 DG-0158', '2024-12-16 DG-0158', '2024-12-17 DG-0158']

In [4]:
OPS1['OPS1-CPH-d5'] = pd.concat([OPS1['OPS1-CPH-d5-1'], OPS1['OPS1-CPH-d5-2']], ignore_index=True)
OPS1['OPS1-NP-d2'] = pd.concat([OPS1['OPS1-NP-d2-1'], OPS1['OPS1-NP-d2-2']], ignore_index=True)
OPS2['OPS2-CPH-d3'] = pd.concat([OPS2['OPS2-CPH-d3-1'], OPS2['OPS2-CPH-d3-2']], ignore_index=True)
OPS2['OPS2-CPH-d3'] = pd.concat([OPS2['OPS2-CPH-d3'], OPS2['OPS2-CPH-d3-3']], ignore_index=True)

NS1_bg_mean, NS1_bg_std, NS1_bg_error = bin_mean(['2024-12-06 11:05:00', '2024-12-06 11:45:00'], 
                                              NS1['2024-12-06_NanoScan'], NS_bins, 'Time', 0.1)
for key in NS_dict_keys:
    df = NS1[key]
    for i, bin in enumerate(NS_bins):
        df[bin] = df[bin] - NS1_bg_mean[i]
    NS1[key] = df

NS2_bg_mean, NS2_bg_std, NS2_bg_error = bin_mean(['2024-12-06 11:05:00', '2024-12-06 11:45:00'], 
                                              NS2['2024-12-06_NanoScan'], NS_bins, 'Time', 0.1)
for key in NS_dict_keys:
    df = NS2[key]
    for i, bin in enumerate(NS_bins):
        df[bin] = df[bin] - NS2_bg_mean[i]
    NS2[key] = df

for key in ACSM_keys:
    ACSM[key] = ACSM[key][(ACSM[key]['Org'] >= 0) & (ACSM[key]['Org'] <= 50)]

OPS_mid_point = [0.337, 0.419, 0.522, 0.650, 0.809, 1.007, 1.254, 1.562, 1.944, 2.421, 3.014, 3.752, 4.672, 5.816, 7.241, 9.015]

OPS1_mass = {}
for key in OPS1_dict_keys:
    mass_conc = calc_mass_conc(OPS1[key], OPS_keys, OPS_mid_point, 1.2)
    OPS1_mass[key] = mass_conc

OPS2_mass = {}
for key in OPS2_dict_keys:
    mass_conc = calc_mass_conc(OPS2[key], OPS_keys, OPS_mid_point, 1.2)
    OPS2_mass[key] = mass_conc

NS_bin_mean = []
for key in NS_bins:
    NS_bin_mean.append(float(key) / 1000)

NS1_mass = {}
NS2_mass = {}
for key in NS_dict_keys:
    mass_conc1 = calc_mass_conc(NS1[key], NS_bins, NS_bin_mean, 1.2)
    NS1_mass[key] = mass_conc1
    mass_conc2 = calc_mass_conc(NS2[key], NS_bins, NS_bin_mean, 1.2)
    NS2_mass[key] = mass_conc2

In [5]:
timestamps = [['2024-12-09 12:00:00', '2024-12-09 16:30:00'],
              ['2024-12-10 09:15:00', '2024-12-10 16:00:00'],
              ['2024-12-11 09:00:00', '2024-12-11 16:00:00'],
              ['2024-12-12 09:00:00', '2024-12-12 16:00:00'],
              ['2024-12-13 08:45:00', '2024-12-13 14:00:00'],
              ['2024-12-16 09:45:00', '2024-12-16 15:45:00'],
              ['2024-12-17 09:20:00', '2024-12-17 15:45:00']]

In [7]:
for i, times in enumerate(timestamps):
    print(times[0].split(' ')[0])

    start, end = pd.to_datetime(times[0]), pd.to_datetime(times[1])

    # NanoScan, number
    df_NS1 = NS1[NS_dict_keys[i]][(NS1[NS_dict_keys[i]]['Time'] >= start) & (NS1[NS_dict_keys[i]]['Time'] <= end)]
    df_NS2 = NS2[NS_dict_keys[i]][(NS2[NS_dict_keys[i]]['Time'] >= start) & (NS2[NS_dict_keys[i]]['Time'] <= end)]

    PN01_array_NS1 = df_NS1.iloc[:,3:12].sum(axis = 1)
    PN03_array_NS1 = df_NS1.iloc[:,3:15].sum(axis = 1)
    PN01_NS1, PN01_STD_NS1 = PN01_array_NS1.mean(), PN01_array_NS1.std()/np.sqrt(len(PN01_array_NS1))
    PN03_NS1, PN03_STD_NS1 = PN03_array_NS1.mean(), PN03_array_NS1.std()/np.sqrt(len(PN03_array_NS1))
    print(f'Station 1, NanoScan (cm^-3):   PN0.1 = {PN01_NS1:.2f}+-{PN01_STD_NS1:.2f} and PN0.3 = {PN03_NS1:.2f}+-{PN03_STD_NS1:.2f}')

    PN01_array_NS2 = df_NS2.iloc[:,3:12].sum(axis = 1)
    PN03_array_NS2 = df_NS2.iloc[:,3:15].sum(axis = 1)
    PN01_NS2, PN01_STD_NS2 = PN01_array_NS2.mean(), PN01_array_NS2.std()/np.sqrt(len(PN01_array_NS2))
    PN03_NS2, PN03_STD_NS2 = PN03_array_NS2.mean(), PN03_array_NS2.std()/np.sqrt(len(PN03_array_NS2))
    print(f'Station 2, NanoScan (cm^-3):   PN0.1 = {PN01_NS2:.2f}+-{PN01_STD_NS2:.2f} and PN0.3 = {PN03_NS2:.2f}+-{PN03_STD_NS2:.2f}')

    # NanoScan, mass
    df_NS1_mass = NS1_mass[NS_dict_keys[i]][(NS1_mass[NS_dict_keys[i]]['Time'] >= start) & (NS1_mass[NS_dict_keys[i]]['Time'] <= end)]
    df_NS2_mass = NS2_mass[NS_dict_keys[i]][(NS2_mass[NS_dict_keys[i]]['Time'] >= start) & (NS2_mass[NS_dict_keys[i]]['Time'] <= end)]

    PN01_array_NS1_mass = df_NS1_mass.iloc[:,1:10].sum(axis = 1)
    PN03_array_NS1_mass = df_NS1_mass.iloc[:,1:13].sum(axis = 1)
    PN01_NS1_mass, PN01_STD_NS1_mass = PN01_array_NS1_mass.mean(), PN01_array_NS1_mass.std()/np.sqrt(len(PN01_array_NS1_mass))
    PN03_NS1_mass, PN03_STD_NS1_mass = PN03_array_NS1_mass.mean(), PN03_array_NS1_mass.std()/np.sqrt(len(PN03_array_NS1_mass))
    # print(f'Station 1, NanoScan (ug/m^3):   PM0.1 = {PN01_NS1_mass:.2f}+-{PN01_STD_NS1_mass:.2f} and PM0.3 = {PN03_NS1_mass:.2f}+-{PN03_STD_NS1_mass:.2f}')

    PN01_array_NS2_mass = df_NS2_mass.iloc[:,1:10].sum(axis = 1)
    PN03_array_NS2_mass = df_NS2_mass.iloc[:,1:13].sum(axis = 1)
    PN01_NS2_mass, PN01_STD_NS2_mass = PN01_array_NS2_mass.mean(), PN01_array_NS2_mass.std()/np.sqrt(len(PN01_array_NS2_mass))
    PN03_NS2_mass, PN03_STD_NS2_mass = PN03_array_NS2_mass.mean(), PN03_array_NS2_mass.std()/np.sqrt(len(PN03_array_NS2_mass))
    # print(f'Station 2, NanoScan (ug/m^3):   PM0.1 = {PN01_NS2_mass:.2f}+-{PN01_STD_NS2_mass:.2f} and PM0.3 = {PN03_NS2_mass:.2f}+-{PN03_STD_NS2_mass:.2f}')

    # OPS, number
    df_OPS1 = OPS1[OPS1_dict_keys[i]][(OPS1[OPS1_dict_keys[i]]['Time'] >= start) & (OPS1[OPS1_dict_keys[i]]['Time'] <= end)]
    df_OPS2 = OPS2[OPS2_dict_keys[i]][(OPS2[OPS2_dict_keys[i]]['Time'] >= start) & (OPS2[OPS2_dict_keys[i]]['Time'] <= end)]

    PN1_array_OPS1 = df_OPS1.iloc[:,1:7].sum(axis = 1)
    PN1_OPS1, PN1_STD_OPS1 = PN1_array_OPS1.mean(), PN1_array_OPS1.std()/np.sqrt(len(PN1_array_OPS1))
    PN25_OPS1, PN25_STD_OPS1 = df_OPS1['PN2.5'].mean(), df_OPS1['PN2.5'].std()/np.sqrt(len(df_OPS1['PN2.5']))
    PN10_OPS1, PN10_STD_OPS1 = df_OPS1['Total Conc'].mean(), df_OPS1['Total Conc'].std()/np.sqrt(len(df_OPS1['Total Conc']))
    # print(f'Station 1, OPS (cm^-3):   PN1 = {PN1_OPS1:.2f}+-{PN1_STD_OPS1:.2f} and PN2.5 = {PN25_OPS1:.2f}+-{PN25_STD_OPS1:.2f} and PN10 = {PN10_OPS1:.2f}+-{PN10_STD_OPS1:.2f}')

    PN1_array_OPS2 = df_OPS2.iloc[:,1:7].sum(axis = 1)
    PN1_OPS2, PN1_STD_OPS2 = PN1_array_OPS2.mean(), PN1_array_OPS2.std()/np.sqrt(len(PN1_array_OPS2))
    PN25_OPS2, PN25_STD_OPS2 = df_OPS2['PN2.5'].mean(), df_OPS2['PN2.5'].std()/np.sqrt(len(df_OPS2['PN2.5']))
    PN10_OPS2, PN10_STD_OPS2 = df_OPS2['Total Conc'].mean(), df_OPS2['Total Conc'].std()/np.sqrt(len(df_OPS2['Total Conc']))
    # print(f'Station 2, OPS (cm^-3):   PN1 = {PN1_OPS2:.2f}+-{PN1_STD_OPS2:.2f} and PN2.5 = {PN25_OPS2:.2f}+-{PN25_STD_OPS2:.2f} and PN10 = {PN10_OPS2:.2f}+-{PN10_STD_OPS2:.2f}')

    # OPS, mass
    df_OPS1_mass = OPS1_mass[OPS1_dict_keys[i]][(OPS1_mass[OPS1_dict_keys[i]]['Time'] >= start) & (OPS1_mass[OPS1_dict_keys[i]]['Time'] <= end)]
    df_OPS2_mass = OPS2_mass[OPS2_dict_keys[i]][(OPS2_mass[OPS2_dict_keys[i]]['Time'] >= start) & (OPS2_mass[OPS2_dict_keys[i]]['Time'] <= end)]

    PN1_array_OPS1_mass = df_OPS1_mass.iloc[:,1:7].sum(axis = 1)
    PN25_array_OPS1_mass = df_OPS1_mass.iloc[:,1:11].sum(axis = 1)
    PN10_array_OPS1_mass = df_OPS1_mass.iloc[:,1:17].sum(axis = 1)
    PN1_OPS1_mass, PN1_STD_OPS1_mass = PN1_array_OPS1_mass.mean(), PN1_array_OPS1_mass.std()/np.sqrt(len(PN1_array_OPS1_mass))
    PN25_OPS1_mass, PN25_STD_OPS1_mass = PN25_array_OPS1_mass.mean(), PN25_array_OPS1_mass.std()/np.sqrt(len(PN25_array_OPS1_mass))
    PN10_OPS1_mass, PN10_STD_OPS1_mass = PN10_array_OPS1_mass.mean(), PN10_array_OPS1_mass.std()/np.sqrt(len(PN10_array_OPS1_mass))
    # print(f'Station 1, OPS (ug/m^3):   PM1 = {PN1_OPS1_mass:.2f}+-{PN1_STD_OPS1_mass:.2f} and PM2.5 = {PN25_OPS1_mass:.2f}+-{PN25_STD_OPS1_mass:.2f} and PM10 = {PN10_OPS1_mass:.2f}+-{PN10_STD_OPS1_mass:.2f}')

    PM25_NSOPS1 = PN25_OPS1_mass + PN03_NS1_mass
    PN25_NSOPS1 = PN25_OPS1 + PN03_NS1
    PM10_NSOPS1 = PM25_NSOPS1 + PN10_OPS1_mass
    PN10_NSOPS1 = PN25_NSOPS1 + PN10_OPS1
    print(f'Station 1, OPS+NS: PN2.5 = {PN25_NSOPS1:.3f} and PM2.5 = {PM25_NSOPS1:.3f}')
    print(f'Station 1, OPS+NS: PN10 = {PN10_NSOPS1:.3f} and PM10 = {PM10_NSOPS1:.3f}')

    PN1_array_OPS2_mass = df_OPS2_mass.iloc[:,1:7].sum(axis = 1)
    PN25_array_OPS2_mass = df_OPS2_mass.iloc[:,1:11].sum(axis = 1)
    PN10_array_OPS2_mass = df_OPS2_mass.iloc[:,1:17].sum(axis = 1)
    PN1_OPS2_mass, PN1_STD_OPS2_mass = PN1_array_OPS2_mass.mean(), PN1_array_OPS2_mass.std()/np.sqrt(len(PN1_array_OPS2_mass))
    PN25_OPS2_mass, PN25_STD_OPS2_mass = PN25_array_OPS2_mass.mean(), PN25_array_OPS2_mass.std()/np.sqrt(len(PN25_array_OPS2_mass))
    PN10_OPS2_mass, PN10_STD_OPS2_mass = PN10_array_OPS2_mass.mean(), PN10_array_OPS2_mass.std()/np.sqrt(len(PN10_array_OPS2_mass))
    # print(f'Station 1, OPS (ug/m^3):   PM1 = {PN1_OPS2_mass:.2f}+-{PN1_STD_OPS2_mass:.2f} and PM2.5 = {PN25_OPS2_mass:.2f}+-{PN25_STD_OPS2_mass:.2f} and PM10 = {PN10_OPS2_mass:.2f}+-{PN10_STD_OPS2_mass:.2f}')

    PM25_NSOPS2 = PN25_OPS2_mass + PN03_NS2_mass
    PN25_NSOPS2 = PN25_OPS2 + PN03_NS2
    PM10_NSOPS2 = PM25_NSOPS2 + PN10_OPS2_mass
    PN10_NSOPS2 = PN25_NSOPS2 + PN10_OPS2
    print(f'Station 2, OPS+NS: PN2.5 = {PN25_NSOPS2:.3f} and PM2.5 = {PM25_NSOPS2:.3f}')
    print(f'Station 2, OPS+NS: PN10 = {PN10_NSOPS2:.3f} and PM10 = {PM10_NSOPS2:.3f}')

    # LCS
    df_LCS1 = LCS[LCS1_dict_keys[i]][(LCS[LCS1_dict_keys[i]]['Time'] >= start) & (LCS[LCS1_dict_keys[i]]['Time'] <= end)]
    df_LCS2 = LCS[LCS2_dict_keys[i]][(LCS[LCS2_dict_keys[i]]['Time'] >= start) & (LCS[LCS2_dict_keys[i]]['Time'] <= end)]
    df_LCS2 = df_LCS2.dropna()

    PN1_LCS1, PN1_STD_LCS1 = df_LCS1['PN<1'].mean(), df_LCS1['PN<1'].std()/np.sqrt(len(df_LCS1['PN<1']))
    PN25_LCS1, PN25_STD_LCS1 = df_LCS1['PN<2.5'].mean(), df_LCS1['PN<2.5'].std()/np.sqrt(len(df_LCS1['PN<2.5']))
    PN10_LCS1, PN10_STD_LCS1 = df_LCS1['PN<10'].mean(), df_LCS1['PN<10'].std()/np.sqrt(len(df_LCS1['PN<10']))
    print(f'Station 1, LCS (DG-0159) (cm^-3):   PN1 = {PN1_LCS1:.2f}+-{PN1_STD_LCS1:.2f} and PN2.5 = {PN25_LCS1:.2f}+-{PN25_STD_LCS1:.2f} and PN10 = {PN10_LCS1:.2f}+-{PN10_STD_LCS1:.2f}')

    PN1_LCS2, PN1_STD_LCS2 = df_LCS2['PN<1'].mean(), df_LCS2['PN<1'].std()/np.sqrt(len(df_LCS2['PN<1']))
    PN25_LCS2, PN25_STD_LCS2 = df_LCS2['PN<2.5'].mean(), df_LCS2['PN<2.5'].std()/np.sqrt(len(df_LCS2['PN<2.5']))
    PN10_LCS2, PN10_STD_LCS2 = df_LCS2['PN<10'].mean(), df_LCS2['PN<10'].std()/np.sqrt(len(df_LCS2['PN<10']))
    print(f'Station 2, LCS (DG-0158) (cm^-3):   PN1 = {PN1_LCS2:.2f}+-{PN1_STD_LCS2:.2f} and PN2.5 = {PN25_LCS2:.2f}+-{PN25_STD_LCS2:.2f} and PN10 = {PN10_LCS2:.2f}+-{PN10_STD_LCS2:.2f}')

    # MA200
    df_MA1 = MA1[MA1_dict_keys[i]][(MA1[MA1_dict_keys[i]]['Time'] >= start) & (MA1[MA1_dict_keys[i]]['Time'] <= end)]
    df_MA2 = MA2[MA2_dict_keys[i]][(MA2[MA2_dict_keys[i]]['Time'] >= start) & (MA2[MA2_dict_keys[i]]['Time'] <= end)]

    MA1_mean, MA1_error = df_MA1['IR BCc'].mean(), df_MA1['IR BCc'].std()/np.sqrt(len(df_MA1['IR BCc']))
    print(f'Station 1, Black carbon (ug/m^3): {MA1_mean:.2f}+-{MA1_error:.2f}')

    MA2_mean, MA2_error = df_MA2['IR BCc'].mean(), df_MA2['IR BCc'].std()/np.sqrt(len(df_MA2['IR BCc']))
    print(f'Station 2, Black carbon (ug/m^3): {MA2_mean:.2f}+-{MA2_error:.2f}')

    # ACSM
    if 0 < i < 5:
        df_ACSM = ACSM[ACSM_keys[i-1]][(ACSM[ACSM_keys[i-1]]['Time'] >= start) & (ACSM[ACSM_keys[i-1]]['Time'] <= end)]

        NH4_mean, NH4_STD = df_ACSM['NH4'].mean(), df_ACSM['NH4'].std()/np.sqrt(len(df_ACSM['NH4']))
        NO3_mean, NO3_STD = df_ACSM['NO3'].mean(), df_ACSM['NO3'].std()/np.sqrt(len(df_ACSM['NO3']))
        # Chl_mean, Chl_STD = df_ACSM['Chl'].mean(), df_ACSM['Chl'].std()/np.sqrt(len(df_ACSM['Chl']))
        SO4_mean, SO4_STD = df_ACSM['SO4'].mean(), df_ACSM['SO4'].std()/np.sqrt(len(df_ACSM['SO4']))
        Org_mean, Org_STD = df_ACSM['Org'].mean(), df_ACSM['Org'].std()/np.sqrt(len(df_ACSM['Org']))

        print(f'ACSM (ug/m^3): NH4 = {NH4_mean:.2f}+-{NH4_STD:.2f} and NO3 = {NO3_mean:.2f}+-{NO3_STD:.2f} and SO4 = {SO4_mean:.2f}+-{SO4_STD:.2f}')     #  and Chl = {Chl_mean:.2f}+-{Chl_STD:.2f}
        print(f'ACSM, Organic matter (ug/m^3): {Org_mean:.2f}+-{Org_STD:.2f}')

        ratio = PM25_NSOPS1 / Org_mean
        print(f'Station 1: Ratio = {ratio:.3f}')

2024-12-09
Station 1, NanoScan (cm^-3):   PN0.1 = 18409.31+-326.67 and PN0.3 = 19864.28+-368.46
Station 2, NanoScan (cm^-3):   PN0.1 = 16530.90+-539.37 and PN0.3 = 17012.66+-557.73
Station 1, OPS+NS: PN2.5 = 19931.582 and PM2.5 = 13.261
Station 1, OPS+NS: PN10 = 19998.982 and PM10 = 24.713
Station 2, OPS+NS: PN2.5 = 17056.522 and PM2.5 = 7.031
Station 2, OPS+NS: PN10 = 17100.447 and PM10 = 14.490
Station 1, LCS (DG-0159) (cm^-3):   PN1 = 66.96+-2.25 and PN2.5 = 67.31+-2.26 and PN10 = 67.38+-2.26
Station 2, LCS (DG-0158) (cm^-3):   PN1 = 51.51+-2.30 and PN2.5 = 51.80+-2.31 and PN10 = 51.85+-2.32
Station 1, Black carbon (ug/m^3): nan+-nan
Station 2, Black carbon (ug/m^3): 0.17+-0.01
2024-12-10
Station 1, NanoScan (cm^-3):   PN0.1 = 16230.29+-326.65 and PN0.3 = 17651.63+-337.52
Station 2, NanoScan (cm^-3):   PN0.1 = 18045.76+-812.99 and PN0.3 = 19104.38+-840.80
Station 1, OPS+NS: PN2.5 = 17703.794 and PM2.5 = 12.471
Station 1, OPS+NS: PN10 = 17756.105 and PM10 = 26.773
Station 2, OPS+NS: 

In [11]:
print(((40.37 + 24.07) / 2) - ((36.33 + 27.98) / 2))

print(((71.28 + 60.01) / 2) / ((65.36 + 40.76) / 2))
print(((71.28 + 60.01) / 2) - ((65.36 + 40.76) / 2))

0.06499999999999773
1.2371843196381453
12.584999999999994


In [7]:
for i, times in enumerate(timestamps):
    print(times[0].split(' ')[0])

    start, end = pd.to_datetime(times[0]), pd.to_datetime(times[1])

    df_OPS1 = OPS1[OPS1_dict_keys[i]][(OPS1[OPS1_dict_keys[i]]['Time'] >= start) & (OPS1[OPS1_dict_keys[i]]['Time'] <= end)]
    df_OPS1['Time'] = df_OPS1['Time'].round('60s')
    df_OPS2 = OPS2[OPS2_dict_keys[i]][(OPS2[OPS2_dict_keys[i]]['Time'] >= start) & (OPS2[OPS2_dict_keys[i]]['Time'] <= end)]
    df_OPS2['Time'] = df_OPS2['Time'].round('60s')
    df_LCS1 = LCS[LCS1_dict_keys[i]][(LCS[LCS1_dict_keys[i]]['Time'] >= start) & (LCS[LCS1_dict_keys[i]]['Time'] <= end)]
    df_LCS1['Time'] = df_LCS1['Time'].round('60s')
    df_LCS2 = LCS[LCS2_dict_keys[i]][(LCS[LCS2_dict_keys[i]]['Time'] >= start) & (LCS[LCS2_dict_keys[i]]['Time'] <= end)]
    df_LCS2['Time'] = df_LCS2['Time'].round('60s')

    # Station 1
    merged1 = pd.merge(df_OPS1, df_LCS1, on = 'Time')

    PN1_array_OPS1 = np.array(merged1.iloc[:,1:7].sum(axis = 1))
    MAE1_PN1 = abs(np.array(merged1['PN<1']) - PN1_array_OPS1).mean()
    MAE1_PN25 = abs(np.array(merged1['PN<2.5']) - np.array(merged1['PN2.5'])).mean()
    MAE1_PN10 = abs(np.array(merged1['PN<10']) - np.array(merged1['Total Conc'])).mean()

    print(f'Station 1 (DG-0159): MAE_PN1 = {MAE1_PN1:.2f} and MAE_PN2.5 = {MAE1_PN25:.2f} and MAE_PN10 = {MAE1_PN10:.2f}')

    # Station 2

    merged2 = pd.merge(df_OPS2, df_LCS2, on = 'Time')

    PN1_array_OPS2 = np.array(merged2.iloc[:,1:7].sum(axis = 1))
    MAE2_PN1 = abs(np.array(merged2['PN<1']) - PN1_array_OPS2).mean()
    MAE2_PN25 = abs(np.array(merged2['PN<2.5']) - np.array(merged2['PN2.5'])).mean()
    MAE2_PN10 = abs(np.array(merged2['PN<10']) - np.array(merged2['Total Conc'])).mean()

    print(f'Station 2 (DG-0158): MAE_PN1 = {MAE2_PN1:.2f} and MAE_PN2.5 = {MAE2_PN25:.2f} and MAE_PN10 = {MAE2_PN10:.2f}')


2024-12-09
Station 1 (DG-0159): MAE_PN1 = 3.52 and MAE_PN2.5 = 3.53 and MAE_PN10 = 3.54
Station 2 (DG-0158): MAE_PN1 = 25.50 and MAE_PN2.5 = 25.62 and MAE_PN10 = 25.64
2024-12-10
Station 1 (DG-0159): MAE_PN1 = 3.27 and MAE_PN2.5 = 3.11 and MAE_PN10 = 3.04
Station 2 (DG-0158): MAE_PN1 = 56.40 and MAE_PN2.5 = 56.60 and MAE_PN10 = 56.61
2024-12-11
Station 1 (DG-0159): MAE_PN1 = 6.93 and MAE_PN2.5 = 6.50 and MAE_PN10 = 6.40
Station 2 (DG-0158): MAE_PN1 = 113.05 and MAE_PN2.5 = 113.35 and MAE_PN10 = 113.41
2024-12-12
Station 1 (DG-0159): MAE_PN1 = 8.01 and MAE_PN2.5 = 7.63 and MAE_PN10 = 7.49
Station 2 (DG-0158): MAE_PN1 = 24.51 and MAE_PN2.5 = 24.35 and MAE_PN10 = 24.33
2024-12-13
Station 1 (DG-0159): MAE_PN1 = 29.50 and MAE_PN2.5 = 29.40 and MAE_PN10 = 29.35
Station 2 (DG-0158): MAE_PN1 = 23.86 and MAE_PN2.5 = 23.76 and MAE_PN10 = 23.74
2024-12-16
Station 1 (DG-0159): MAE_PN1 = 10.91 and MAE_PN2.5 = 10.42 and MAE_PN10 = 10.53
Station 2 (DG-0158): MAE_PN1 = nan and MAE_PN2.5 = nan and MAE_